<a href="https://colab.research.google.com/github/britneytota/weatherwise-Nyaradzo-BritineyTota/blob/main/starter_notebook_NyaradzoBritineyTota.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🌦️ WeatherWise – NyaradzoBritineyTota

Welcome to your **WeatherWise** project notebook! This scaffold is designed to help you build your weather advisor app using Python, visualisations, and AI-enhanced development.

---

📄 **Full Assignment Specification**  
See [`ASSIGNMENT.md`](ASSIGNMENT.md) or check the LMS for full details.

📝 **Quick Refresher**  
A one-page summary is available in [`resources/assignment-summary.md`](resources/assignment-summary.md).

---

🧠 **This Notebook Structure is Optional**  
You’re encouraged to reorganise, rename sections, or remove scaffold cells if you prefer — as long as your final version meets the requirements.

✅ You may delete this note before submission.



## 🧰 Setup and Imports

This section imports commonly used packages and installs any additional tools used in the project.

- You may not need all of these unless you're using specific features (e.g. visualisations, advanced prompting).
- The notebook assumes the following packages are **pre-installed** in the provided environment or installable via pip:
  - `requests`, `matplotlib`, `pyinputplus`
  - `fetch-my-weather` (for accessing weather data easily)
  - `hands-on-ai` (for AI logging, comparisons, or prompting tools)

If you're running this notebook in **Google Colab**, uncomment the following lines to install the required packages.


In [ ]:
# %%
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus
# %%

In [ ]:
# Import packages after installation
import os
import requests # Import requests as it is used later in the notebook
import pyinputplus as pyip # Import pyinputplus and alias it as pyip
import matplotlib.pyplot as plt # Import matplotlib.pyplot and alias it as plt

# Set environment variables
os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
# The API key input might need to be handled interactively or set another way
# depending on the execution environment.
# For now, keep it as is, but be aware it might pause execution.
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')

Enter your API key: 2aa72b4fa5db4ed6b41181657252105


## 📦 Setup and Configuration
Import required packages and setup environment.

In [ ]:
import requests
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus
WEATHER_API_KEY = "2aa72b4fa5db4ed6b41181657252105"  # Replace with your actual API key
BASE_WEATHER_URL = "http://api.weatherapi.com/v1"


## 🌤️ Weather Data Functions

In [ ]:
# Define get_weather_data() function here
def get_weather_data(location, forecast_days=3):
    endpoint = f"{BASE_WEATHER_URL}/forecast.json"
    params = {
        'key': WEATHER_API_KEY,
        'q': location,
        'days': forecast_days,
        'aqi': 'no',
        'alerts': 'no'
    }
    try:
        response = requests.get(endpoint, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching data for {location}: {e}")
        return None

## 📊 Visualisation Functions

In [ ]:
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    if weather_data and 'forecast' in weather_data and 'forecastday' in weather_data['forecast']:
        dates = [day['date'] for day in weather_data['forecast']['forecastday']]
        temps = [day['day']['avgtemp_c'] for day in weather_data['forecast']['forecastday']]

        plt.figure(figsize=(10, 5))
        plt.plot(dates, temps, marker='o', linestyle='--', color='tomato', linewidth=2)
        plt.title(f"Average Temperature in {weather_data['location']['name']}", fontsize=14)
        plt.xlabel('Date')
        plt.ylabel('Temperature (°C)')
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        if output_type == 'display':
            plt.show()
        else:
            return plt.gcf()
    else:
        print("Error: Could not extract temperature data.")



In [ ]:

def create_precipitation_visualisation(weather_data, output_type='display'):
    if weather_data and 'forecast' in weather_data and 'forecastday' in weather_data['forecast']:
        dates = [day['date'] for day in weather_data['forecast']['forecastday']]
        precipitation = [day['day']['daily_chance_of_rain'] for day in weather_data['forecast']['forecastday']]

        plt.figure(figsize=(10, 5))
        plt.bar(dates, precipitation, color='skyblue')
        plt.title(f"Chance of Rain in {weather_data['location']['name']}", fontsize=14)
        plt.xlabel('Date')
        plt.ylabel('Chance of Rain (%)')
        plt.grid(axis='y')
        plt.xticks(rotation=45)
        plt.tight_layout()
        if output_type == 'display':
            plt.show()
        else:
            return plt.gcf()
    else:
        print("Error: Could not extract precipitation data.")


## 🤖 Natural Language Processing

In [ ]:
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    question = question.lower()
    parsed = {'location': None, 'attribute': None, 'time_period': 'today'}

    if 'temperature' in question or 'temp' in question:
        parsed['attribute'] = 'temperature'
    elif 'rain' in question or 'precipitation' in question:
        parsed['attribute'] = 'precipitation'

    if 'tomorrow' in question:
        parsed['time_period'] = 'tomorrow'
    elif 'week' in question or '5 days' in question:
        parsed['time_period'] = '5-day'

    tokens = question.split()
    for i in range(len(tokens)):
        if tokens[i] in ['in', 'for', 'at'] and i + 1 < len(tokens):
            parsed['location'] = ' '.join(tokens[i + 1:]).capitalize()
            break
        elif i == 0 and tokens[i].isalpha() and parsed['location'] is None:
            parsed['location'] = tokens[i].capitalize()

    return parsed

def generate_weather_response(parsed_question, weather_data):
    attribute = parsed_question['attribute']
    time_period = parsed_question['time_period']
    location = parsed_question['location']

    if not weather_data:
        return f"Sorry, I couldn't retrieve the weather data for {location}."

    forecast_days = weather_data.get('forecast', {}).get('forecastday', [])
    if not forecast_days:
        return "No forecast data available."

    if time_period == 'today':
        if len(forecast_days) >= 1:
            day_data = forecast_days[0]['day']
            time_desc = "today"
        else:
            return "No data for today."
    elif time_period == 'tomorrow':
        if len(forecast_days) >= 2:
            day_data = forecast_days[1]['day']
            time_desc = "tomorrow"
        else:
            return "No data for tomorrow."
    elif time_period == '5-day':
        response = f"Here is the 5-day forecast for {location}:\n"
        for day in forecast_days:
            date = day['date']
            avg_temp = day['day'].get('avgtemp_c', 'N/A')
            rain_chance = day['day'].get('daily_chance_of_rain', 'N/A')
            response += f"{date}: Avg Temp: {avg_temp}°C, Rain: {rain_chance}%\n"
        return response
    else:
        return "Time period not recognized."

    if attribute == 'temperature':
        return f"The average temperature in {location} {time_desc} is {day_data.get('avgtemp_c', 'N/A')}°C."
    elif attribute == 'precipitation':
        return f"There is a {day_data.get('daily_chance_of_rain', 'N/A')}% chance of rain in {location} {time_desc}."
    else:
        return "I couldn't identify the weather detail you're asking for."

# Define the menu function outside of generate_weather_response
def main_menu():
    print("=== Welcome to Weather Advisor ===")
    last_location = None

    while True:
        print("\nWeather Advisor Menu")
        print("Please select one of the following:")
        choice = pyip.inputMenu(
            ['Ask a question', 'Show temperature chart', 'Show precipitation chart', 'Exit'],
            numbered=True
        )

        if choice == 'Ask a question':
            question = input("What would you like to know about the weather?\n> ")
            parsed = parse_weather_question(question)
            data = get_weather_data(parsed['location'])
            response = generate_weather_response(parsed, data)
            print("\n" + response + "\n")
            last_location = parsed['location']

        elif choice in ['Show temperature chart', 'Show precipitation chart']:
            if last_location:
                use_last = pyip.inputYesNo(f"Use last location ({last_location})? (yes/no): ")
                if use_last == 'yes':
                    location = last_location
                else:
                    location = pyip.inputStr("Enter a location: ")
                    last_location = location
            else:
                location = pyip.inputStr("Enter a location: ")
                last_location = location

            data = get_weather_data(location)
            if data:
                if choice == 'Show temperature chart':
                    create_temperature_visualisation(data)
                else:
                    create_precipitation_visualisation(data)

        elif choice == 'Exit':
            print("Goodbye!")
            break

## 🧭 User Interface

In [ ]:
# Define menu functions using pyinputplus or ipywidgets here
def menu():
    print("\nWeather Advisor Menu")
    while True:
        choice = pyip.inputMenu(['Ask a question', 'Show temperature chart', 'Show precipitation chart', 'Exit'], numbered=True)

        if choice == 'Ask a question':
            question = input("Ask your weather question (e.g., What's the temp in Paris tomorrow?): ")
            parsed = parse_weather_question(question)
            location = parsed['location']
            if location:
                data = get_weather_data(location)
                response = generate_weather_response(parsed, data)
                print(response)
            else:
                print("Please include a location in your question.")

        elif choice == 'Show temperature chart':
            location = input("Enter a location: ")
            data = get_weather_data(location)
            if data:
                create_temperature_visualisation(data)
            else:
                print(f"Could not retrieve data for {location}.")

        elif choice == 'Show precipitation chart':
            location = input("Enter a location: ")
            data = get_weather_data(location)
            if data:
                create_precipitation_visualisation(data)
            else:
                print(f"Could not retrieve data for {location}.")

        elif choice == 'Exit':
            print("Goodbye!")
            break

## 🧩 Main Application Logic

In [ ]:
# %%
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus
# %%
# Import packages after installation
import os
import requests # Import requests as it is used later in the notebook
import pyinputplus as pyip # Import pyinputplus and alias it as pyip
import matplotlib.pyplot as plt # Import matplotlib.pyplot and alias it as plt

# Set environment variables
os.environ['HANDS_ON_AI_SERVER'] = 'http://ollama.serveur.au'
os.environ['HANDS_ON_AI_MODEL'] = 'granite3.2'
# The API key input might need to be handled interactively or set another way
# depending on the execution environment.
# For now, keep it as is, but be aware it might pause execution.
os.environ['HANDS_ON_AI_API_KEY'] = input('Enter your API key: ')
import requests
# 🧪 Optional packages — uncomment if needed in Colab or JupyterHub
!pip install fetch-my-weather
!pip install hands-on-ai
!pip install pyinputplus
WEATHER_API_KEY = "2aa72b4fa5db4ed6b41181657252105"  # Replace with your actual API key
BASE_WEATHER_URL = "http://api.weatherapi.com/v1"
# Define get_weather_data() function here
def get_weather_data(location, forecast_days=3):
    endpoint = f"{BASE_WEATHER_URL}/forecast.json"
    params = {
        'key': WEATHER_API_KEY,
        'q': location,
        'days': forecast_days,
        'aqi': 'no',
        'alerts': 'no'
    }
    try:
        response = requests.get(endpoint, params=params)
        response.raise_for_status()
        return response.json()
    except requests.RequestException as e:
        print(f"Error fetching data for {location}: {e}")
        return None
# Define create_temperature_visualisation() and create_precipitation_visualisation() here
def create_temperature_visualisation(weather_data, output_type='display'):
    if weather_data and 'forecast' in weather_data and 'forecastday' in weather_data['forecast']:
        dates = [day['date'] for day in weather_data['forecast']['forecastday']]
        temps = [day['day']['avgtemp_c'] for day in weather_data['forecast']['forecastday']]

        plt.figure(figsize=(10, 5))
        plt.plot(dates, temps, marker='o', linestyle='--', color='tomato', linewidth=2)
        plt.title(f"Average Temperature in {weather_data['location']['name']}", fontsize=14)
        plt.xlabel('Date')
        plt.ylabel('Temperature (°C)')
        plt.grid(True)
        plt.xticks(rotation=45)
        plt.tight_layout()
        if output_type == 'display':
            plt.show()
        else:
            return plt.gcf()
    else:
        print("Error: Could not extract temperature data.")

def create_precipitation_visualisation(weather_data, output_type='display'):
    if weather_data and 'forecast' in weather_data and 'forecastday' in weather_data['forecast']:
        dates = [day['date'] for day in weather_data['forecast']['forecastday']]
        precipitation = [day['day']['daily_chance_of_rain'] for day in weather_data['forecast']['forecastday']]

        plt.figure(figsize=(10, 5))
        plt.bar(dates, precipitation, color='skyblue')
        plt.title(f"Chance of Rain in {weather_data['location']['name']}", fontsize=14)
        plt.xlabel('Date')
        plt.ylabel('Chance of Rain (%)')
        plt.grid(axis='y')
        plt.xticks(rotation=45)
        plt.tight_layout()
        if output_type == 'display':
            plt.show()
        else:
            return plt.gcf()
    else:
        print("Error: Could not extract precipitation data.")
# Define parse_weather_question() and generate_weather_response() here
def parse_weather_question(question):
    question = question.lower()
    parsed = {'location': None, 'attribute': None, 'time_period': 'today'}

    if 'temperature' in question or 'temp' in question:
        parsed['attribute'] = 'temperature'
    elif 'rain' in question or 'precipitation' in question:
        parsed['attribute'] = 'precipitation'

    if 'tomorrow' in question:
        parsed['time_period'] = 'tomorrow'
    elif 'week' in question or '5 days' in question:
        parsed['time_period'] = '5-day'

    tokens = question.split()
    for i in range(len(tokens)):
        if tokens[i] in ['in', 'for', 'at'] and i + 1 < len(tokens):
            parsed['location'] = ' '.join(tokens[i + 1:]).capitalize()
            break
        elif i == 0 and tokens[i].isalpha() and parsed['location'] is None:
            parsed['location'] = tokens[i].capitalize()

    return parsed

def generate_weather_response(parsed_question, weather_data):
    attribute = parsed_question['attribute']
    time_period = parsed_question['time_period']
    location = parsed_question['location']

    if not weather_data:
        return f"Sorry, I couldn't retrieve the weather data for {location}."

    forecast_days = weather_data.get('forecast', {}).get('forecastday', [])
    if not forecast_days:
        return "No forecast data available."

    if time_period == 'today':
        if len(forecast_days) >= 1:
            day_data = forecast_days[0]['day']
            time_desc = "today"
        else:
            return "No data for today."
    elif time_period == 'tomorrow':
        if len(forecast_days) >= 2:
            day_data = forecast_days[1]['day']
            time_desc = "tomorrow"
        else:
            return "No data for tomorrow."
    elif time_period == '5-day':
        response = f"Here is the 5-day forecast for {location}:\n"
        for day in forecast_days:
            date = day['date']
            avg_temp = day['day'].get('avgtemp_c', 'N/A')
            rain_chance = day['day'].get('daily_chance_of_rain', 'N/A')
            response += f"{date}: Avg Temp: {avg_temp}°C, Rain: {rain_chance}%\n"
        return response
    else:
        return "Time period not recognized."

    if attribute == 'temperature':
        return f"The average temperature in {location} {time_desc} is {day_data.get('avgtemp_c', 'N/A')}°C."
    elif attribute == 'precipitation':
        return f"There is a {day_data.get('daily_chance_of_rain', 'N/A')}% chance of rain in {location} {time_desc}."
    else:
        return "I couldn't identify the weather detail you're asking for."

# Define the menu function outside of generate_weather_response
def main_menu():
    print("=== Welcome to Weather Advisor ===")
    last_location = None

    while True:
        print("\nWeather Advisor Menu")
        print("Please select one of the following:")
        choice = pyip.inputMenu(
            ['Ask a question', 'Show temperature chart', 'Show precipitation chart', 'Exit'],
            numbered=True
        )

        if choice == 'Ask a question':
            question = input("What would you like to know about the weather?\n> ")
            parsed = parse_weather_question(question)
            data = get_weather_data(parsed['location'])
            response = generate_weather_response(parsed, data)
            print("\n" + response + "\n")
            last_location = parsed['location']

        elif choice in ['Show temperature chart', 'Show precipitation chart']:
            if last_location:
                use_last = pyip.inputYesNo(f"Use last location ({last_location})? (yes/no): ")
                if use_last == 'yes':
                    location = last_location
                else:
                    location = pyip.inputStr("Enter a location: ")
                    last_location = location
            else:
                location = pyip.inputStr("Enter a location: ")
                last_location = location

            data = get_weather_data(location)
            if data:
                if choice == 'Show temperature chart':
                    create_temperature_visualisation(data)
                else:
                    create_precipitation_visualisation(data)

        elif choice == 'Exit':
            print("Goodbye!")
            break

## 🧪 Testing and Examples

In [ ]:
## Include sample input/output for each function
def run_tests():
    print("\n--- Running Tests ---")
    sample_location = "London"
    data = get_weather_data(sample_location)
    assert data is not None, "Failed to get weather data."
    assert "forecast" in data, "Missing 'forecast' in response."
    assert "forecastday" in data["forecast"], "Missing 'forecastday' in forecast."
    assert "day" in data["forecast"]["forecastday"][0], "Missing 'day' in forecastday."
    print("All tests passed!")

# === Run the application ===
if __name__ == "__main__":
    run_tests()
    menu()

## 🗂️ AI Prompting Log (Optional)
Add markdown cells here summarising prompts used or link to AI conversations in the `ai-conversations/` folder.